In [1]:
from datetime import date, datetime, timedelta

#### liste des fonctions à créer
- Échange
- Ballotage/Libérer joueur
- Commutations
    - Blessures
- Repêchages

In [1]:
#fonctions de base

#créer la table DRAFT en SQL avec les choix au repêchage
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
#choix = pd.read_csv("C:\dev\pool-de-hockey\src\data\listeChoix.csv", sep=";")
#choix.to_sql('DRAFT', conn, if_exists='replace', index = False)
def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

In [3]:
#DO NOT EXECUTE créer une fausse table ALIGNEMENTS POUR FAIRE LES TESTS
#alignements = pd.read_csv("alignements.csv", sep=";")
#alignements.to_sql('copALIGN', conn, if_exists='replace', index = False)

In [4]:
#Ensuite:
    # - vérificateur de lineup:
        #-serait intéressant d'être capable de le jumeler à la fonction échange et que dans le cas où un échange                rend un lineup non-conforme, un paramètre indique si on continue l'échange quand même (exemple l'équipe             en défaut compte mettre un joueur au ballotage immédiatement après) où on le break (et donc l'annule                avec undoÉchange)

In [11]:
#importer l'échange
elements = pd.read_csv("Trade.csv", sep=";")

#générer tradeJoueurs et idJoueurs
is_joueur = elements["Élément"] == "joueur"
joueurs = elements[is_joueur]
noms = []
n = len(joueurs)
for i in range(0, n):
    joueur = joueurs.iloc[i]
    prenom = joueur["Prénom"]
    nom = joueur["Nom"]
    compl = [prenom, nom]
    noms.__iadd__([compl])
idJoueurs = []
for i in range(0, n):
    joueur = noms[i]
    prenom = joueur[0]
    nom = joueur[1]
    idNHL = getId(prenom, nom)
    idJoueurs.__iadd__([idNHL])
joueurs["idNHL"] = idJoueurs
tradeJoueurs = joueurs[["idNHL", "equipeRecoit"]]

#générer tradeChoix et idChoix
is_choix = elements["Élément"] == "choix"
choix = elements[is_choix]
choix["Année"] = choix["Année"].astype(int)
choix["Ronde"] = choix["Ronde"].astype(int)
idChoix = []
n = len(choix)
for i in range(0, n):
    choixi = choix.iloc[i]
    typeCh = choixi["Type"]
    année = choixi["Année"]
    année = str(année)
    ronde = choixi["Ronde"]
    ronde = str(ronde)
    equ = choixi["original"]
    ide = typeCh + année + ronde + equ
    idChoix.__iadd__([ide])
choix["idChoix"] = idChoix
tradeChoix = choix[["idChoix", "equipeRecoit"]]

print(elements)
print(tradeJoueurs)
print(tradeChoix)

  equipeRecoit Élément Prénom   Nom Type   Année  Ronde original
0          cle  joueur    Cam  York  NaN     NaN    NaN      NaN
1          hub   choix    NaN   NaN  PRO  2021.0    7.0      cle
     idNHL equipeRecoit
0  8481546          cle
       idChoix equipeRecoit
1  PRO20217cle          hub


In [10]:
#voici les fonctions pour faire un échange en se basant sur les csvs

#importer les éléments joueurs de l'échange
#tradeJoueurs = pd.read_csv("TradeJoueurs.csv", sep=";")
#idJoueurs = tradeJoueurs["idNHL"]

#importer les éléments choix de l'échange
#tradeChoix = pd.read_csv("TradeChoix.csv", sep=";")
#idChoix = tradeChoix["idChoix"]

#fonction qui retourne le idNHL d'un joueur
def getId(prenom, nom):
    req = "SELECT idNHL FROM JOUEURS WHERE prenomJoueur == '" + prenom + "' AND nomJoueur == '" + nom + "'"
    df = pd.read_sql(req, conn)
    idNHL = df.iloc[0]["idNHL"]
    return idNHL

#requête qui va chercher la ligne actuelle d'un joueur dans ALIGNEMENTS
def joueurLive(id):
    requete = "SELECT * FROM copALIGN WHERE idNHL == " + id + " AND dateFin == '2100-01-01'"
    df = pd.read_sql(requete, conn)
    return df

#fonction qui termine la période actuelle des joueurs de l'échange
def finirPeriode(joueurs, dat):
    dat = datetime.strptime(dat, "%Y-%m-%d")
    dat = dat - timedelta(days=1)
    dat = dat.strftime("%Y-%m-%d")
    dat = str(dat)
    for i in joueurs:
        str_i = str(i)
        df = joueurLive(str_i)
        idAlign = df.iloc[0]["idAlignement"]
        idAlign = str(idAlign)
        req = "UPDATE copALIGN SET dateFin = '" + dat + "' WHERE idAlignement == " + idAlign
        execSQL(req, conn, write=True)


#écrire la nouvelle ligne du joueur
def nouvellePeriode(id, dat):
    strId = str(id)
        # idAlignement : max de idAlignement + 1
    reqIdAlign = "SELECT MAX(idAlignement) FROM copALIGN"
    W_idAlign = pd.read_sql(reqIdAlign, conn)
    W_idAlign = W_idAlign.iloc[0]["MAX(idAlignement)"] + 1
        # idPooler : aller le chercher dans TradeJoueurs.csv avec idNHL
    is_idNHL = tradeJoueurs["idNHL"] == id
    joueurTrade = tradeJoueurs[is_idNHL]
    W_idPooler = joueurTrade.iloc[0]["equipeRecoit"]
        # prenomJoueur : aller le chercher dans JOUEURS avec idNHL
    reqPrenom = "SELECT prenomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_prenom = pd.read_sql(reqPrenom, conn)
    W_prenom = W_prenom.iloc[0]["prenomJoueur"]
        # nomJoueur : aller le chercher dans JOUEURS avec idNHL
    reqNom = "SELECT nomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_nom = pd.read_sql(reqNom, conn)
    W_nom = W_nom.iloc[0]["nomJoueur"]
        # idNHL : paramètre de la fonction
        # statutJoueur : aller le chercher dans joueurLive(id)
    df = joueurLive(strId)
    W_statut = df.iloc[0]["statutJoueur"]
        # dateDebut : paramètre de la fonction
        # dateFin : '2100-01-01'
    W_dateFin = "2100-01-01"

    d = {"idAlignement" : [W_idAlign], "idPooler" : [W_idPooler], "prenomJoueur" : [W_prenom], "nomJoueur" : [W_nom], "idNHL" : [id], "statutJoueur" : [W_statut], "dateDebut" : [dat], "dateFin" : [W_dateFin], "pjActuels" : [0], "butsActuels" : [0], "assistActuels" : [0], "pointsActuels": [0]}
    periode = pd.DataFrame(data=d)
    periode.to_sql("copALIGN", conn, if_exists="append", index=False)
    ok = W_prenom + " " + W_nom
    return ok

#fonction qui switch les choix
def switchChoix(idCh):
    is_choix = tradeChoix["idChoix"] == idCh
    choix = tradeChoix[is_choix]
    equ = choix.iloc[0]["equipeRecoit"]
    reqChoix = "UPDATE copDRAFT SET Actuel = '" + equ + "' WHERE idChoix == '" + idCh + "' "
    execSQL(reqChoix, conn, write=True)
    ok = "ok"
    return reqChoix

#fonction get noms de joueur et get choix
def getNom(id):
    strId = str(id)
    reqPrenom = "SELECT prenomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_prenom = pd.read_sql(reqPrenom, conn)
    W_prenom = W_prenom.iloc[0]["prenomJoueur"]
    reqNom = "SELECT nomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_nom = pd.read_sql(reqNom, conn)
    W_nom = W_nom.iloc[0]["nomJoueur"]
    return W_prenom + " " + W_nom

def getChoix(idChoix):
    reqType = "SELECT Type FROM copDRAFT WHERE idChoix = '" + idChoix + "'"
    W_type = pd.read_sql(reqType, conn)
    W_type = W_type.iloc[0]["Type"]
    reqAn = "SELECT Année FROM copDRAFT WHERE idChoix = '" + idChoix + "'"
    W_an = pd.read_sql(reqAn, conn)
    W_an = W_an.iloc[0]["Année"]
    W_an = str(W_an)
    reqRonde = "SELECT Ronde FROM copDRAFT WHERE idChoix = '" + idChoix + "'"
    W_ron = pd.read_sql(reqRonde, conn)
    W_ron = W_ron.iloc[0]["Ronde"]
    W_ron = str(W_ron)
    reqEqu = "SELECT Original FROM copDRAFT WHERE idChoix = '" + idChoix + "'"
    W_Equ = pd.read_sql(reqEqu, conn)
    W_Equ = W_Equ.iloc[0]["Original"]
    return "Ronde " + W_ron + " - " + W_type + " " + W_an + " (" + W_Equ + ")"

#ajouter l'échange au registre
def ajouterRegistre(dat, condition):
    #générer le idTrade
    req = "SELECT MAX(idTrade) FROM ÉCHANGES"
    m = pd.read_sql(req, conn)
    W_idTrade = m.iloc[0]["MAX(idTrade)"] + 1
    #générer EquipeA
    equipes = elements["equipeRecoit"].unique()
    equipeA = equipes[0]
    #générer JoueursA
    is_equA = tradeJoueurs["equipeRecoit"] == equipeA
    joueursA = tradeJoueurs[is_equA]
    joueursA = joueursA["idNHL"]
    a = []
    for i in joueursA:
        nom = getNom(i)
        a.__iadd__([nom])
    str_a = ', '.join([str(elem) for elem in a])
    #générer EquipeB
    equipeB = equipes[1]
    #générer JoueursB
    is_equB = tradeJoueurs["equipeRecoit"] == equipeB
    joueursB = tradeJoueurs[is_equB]
    joueursB = joueursB["idNHL"]
    b = []
    for i in joueursB:
        nom = getNom(i)
        b.__iadd__([nom])
    str_b = ', '.join([str(elem) for elem in b])
    #générer ChoixA
    is_equA = tradeChoix["equipeRecoit"] == equipeA
    choixA = tradeChoix[is_equA]
    choixA = choixA["idChoix"]
    aC = []
    for i in choixA:
        choix = getChoix(i)
        aC.__iadd__([choix])
    str_aC = ', '.join([str(elem) for elem in aC])
    #générer ChoixB
    is_equB = tradeChoix["equipeRecoit"] == equipeB
    choixB = tradeChoix[is_equB]
    choixB = choixB["idChoix"]
    bC = []
    for i in choixB:
        choix = getChoix(i)
        bC.__iadd__([choix])
    str_bC = ', '.join([str(elem) for elem in bC])

    d = {"idTrade" : [W_idTrade], "DateTrade" : [dat], "EquipeA" : [equipeA], "JoueursA" : [str_a], "ChoixA" : [str_aC], "EquipeB" : [equipeB], "JoueursB" : [str_b], "ChoixB" : [str_bC], "Condition" : [condition]}
    echange = pd.DataFrame(data=d)
    echange.to_sql("ÉCHANGES", conn, if_exists="append", index=False)
    ok = "ok"
    return echange

def echange(joueurs, choix, dat, condition="vide"):
    for i in joueurs:
        nouvellePeriode(i, dat)
    finirPeriode(joueurs, dat)
    for i in choix:
        switchChoix(i)
    done = "done"
    ajouterRegistre(dat, condition)
    return done

In [7]:
dat = "2021-02-27"

echange(idJoueurs, idChoix, dat)

'done'

In [8]:
#la fonction undoÉchange()

def IdsJoueursÉchange(idTrade, equipe):
    idTrade = str(idTrade)
    req = "SELECT Joueurs" + equipe + " FROM ÉCHANGES WHERE idTrade == " + idTrade
    joueurs = pd.read_sql(req, conn)
    joueurs = joueurs.iloc[0]["Joueurs" + equipe]
    joueurs = joueurs.split(sep=",")
    n = len(joueurs)
    J = []
    I = []
    for i in range(0, n):
        joueur = joueurs[i]
        joueur = joueur.split()
        J.__iadd__([joueur])
    try:
        for i in range(0, n):
            prenom = J[i][0]
            nom = J[i][1]
            idNHL = getId(prenom, nom)
            I.__iadd__([idNHL])
    except:
        I = []
    return I

def getDateTrade(idTrade):
    strTr = str(idTrade)
    req = "SELECT DateTrade FROM ÉCHANGES WHERE idTrade == " + strTr
    df = pd.read_sql(req, conn)
    dat = df.iloc[0]["DateTrade"]
    return dat

def suppActuel(idTrade, equipe):
    s = IdsJoueursÉchange(idTrade, equipe)
    for i in s:
        stri = str(i) 
        req = "DELETE FROM copALIGN WHERE idNHL == " + stri + " AND dateFIN == '2100-01-01'"
        execSQL(req, conn, write=True)

def updDateFin(idTrade, equipe):
    joueurs = IdsJoueursÉchange(idTrade, equipe)
    dat = getDateTrade(idTrade)
    dat = datetime.strptime(dat, "%Y-%m-%d")
    hier = dat - timedelta(days=1)
    hier = hier.strftime("%Y-%m-%d")
    for i in joueurs:
        stri = str(i)
        upd = "UPDATE copALIGN SET dateFin = '2100-01-01' WHERE dateFin = '" + hier + "' AND idNHL = " + stri
        execSQL(upd, conn, write=True)

def IdsChoixÉchange(idTrade, equipe):
    idTrade = str(idTrade)
    req = "SELECT Choix" + equipe + " FROM ÉCHANGES WHERE idTrade == " + idTrade
    choix = pd.read_sql(req, conn)
    choix = choix.iloc[0]["Choix" + equipe]
    C = []
    try:
        choix = choix.split(sep=", ")
        n = len(choix)
        try:
            for i in range(0, n):
                choixi = choix[i]
                equ = choixi[-4:-1]
                choixi = choixi[6:-1]
                ronde = choixi[0]
                typeAn = choixi[4:-5]
                s = typeAn + ronde + equ
                idChoix = s.replace(" ", "")
                C.__iadd__([idChoix])
        except:
            C = []
    except:
        C = []
    return C

def updChoix(idTrade, equipe):
    strId = str(idTrade)
    choix = IdsChoixÉchange(idTrade, equipe)
    if equipe == "A":
        orig = "B"
    elif equipe == "B":
        orig = "A"
    req = "SELECT Equipe" + orig + " FROM ÉCHANGES WHERE idTrade == " + strId
    equ = pd.read_sql(req, conn)
    equ = equ.iloc[0]["Equipe" + orig]
    for i in choix:
        upd = "UPDATE copDRAFT SET Actuel = '" + equ + "' WHERE idChoix == '" + i + "'"
        execSQL(upd, conn, write=True)

def suppRegistre(idTrade):
    stri = str(idTrade)
    req = "DELETE FROM ÉCHANGES WHERE idTrade == " + stri
    execSQL(req, conn, write=True)

def undoÉchange(idTrade):
    AeB = ["A", "B"]
    for i in AeB:
        suppActuel(idTrade, i)
        updDateFin(idTrade, i)
        updChoix(idTrade, i)
    suppRegistre(idTrade)

In [9]:
undoÉchange(6)